In [ ]:
# pip install langchain chromadb ollama langchain-community langchain_ollama pypdf pdfplumber streamlit psycopg2

In [1]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] =   # Set your Gemini API key here

class CustomEmbeddingFunction:
    def __init__(self, model_name):
        self.embedding_model = GoogleGenerativeAIEmbeddings(model=model_name)

    def __call__(self, input):
        return self.embedding_model.embed_documents(input)

def get_embedding_function(model_name):
    embeddings = CustomEmbeddingFunction(model_name)
    return embeddings

In [2]:
import chromadb
CHROMA_DB_DIR = "./chroma_db"
chroma_client = chromadb.PersistentClient(path=CHROMA_DB_DIR)

# chroma_client.delete_collection(name="my_database")
# Set up Chroma as the vector store
collection = chroma_client.get_or_create_collection(
    name="my_database", metadata={"hnsw:space": "cosine"}, embedding_function=get_embedding_function("models/text-embedding-004")
)

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Read the SQL schema text file
with open("HR postgres.txt", "r", encoding="utf-8") as file:
    sql_schema = file.read()

# Initialize RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # Adjust chunk size as needed
    chunk_overlap=50  # Overlap for better context continuity
)

# Split text into chunks
chunks = text_splitter.split_text(sql_schema)  # Pass raw text directly

# Print the chunks
for i, chunk in enumerate(chunks, 1):
    print(f"🔹 Chunk {i}:\n{chunk}\n{'='*50}")


🔹 Chunk 1:
CREATE SCHEMA hr;

CREATE TABLE hr.departments (
    department_id SERIAL PRIMARY KEY,
    department_name VARCHAR(100) NOT NULL,
    location VARCHAR(100)
);
🔹 Chunk 2:
CREATE TABLE hr.job_roles (
    job_id SERIAL PRIMARY KEY,
    job_title VARCHAR(100) NOT NULL,
    min_salary DECIMAL(10,2),
    max_salary DECIMAL(10,2)
);
🔹 Chunk 3:
CREATE TABLE hr.employees (
    employee_id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
🔹 Chunk 4:
email VARCHAR(100) UNIQUE NOT NULL,
    phone_number VARCHAR(20),
    hire_date DATE NOT NULL,
    job_id INT REFERENCES hr.job_roles(job_id),
🔹 Chunk 5:
job_id INT REFERENCES hr.job_roles(job_id),
    department_id INT REFERENCES hr.departments(department_id),
    salary DECIMAL(10,2) NOT NULL
);


In [4]:
# Collect all chunks first
chunk_ids = [f"chunk_{i}" for i in range(len(chunks))]
chunk_texts = chunks  # List of chunk texts

# Add all chunks at once (batch insert)
collection.add(ids=chunk_ids, documents=chunk_texts)
print(f"✅ Successfully added {len(chunks)} chunks to ChromaDB!")

✅ Successfully added 5 chunks to ChromaDB!


In [17]:
import psycopg2

# Database connection parameters
postgres_ip = "localhost"  # Use only the IP or hostname (remove 'http://')
postgres_port = "5432"
database = "local"
user = "postgres"
password = "postgres"

# Establish connection
try:
    conn = psycopg2.connect(
        host=postgres_ip,
        port=postgres_port,
        database=database,
        user=user,
        password=password
    )
    cursor = conn.cursor()
    
    # Example SQL execution
    cursor.execute("SELECT * FROM hr.employees;")
    rows = cursor.fetchall()

    # Print results
    for row in rows:
        print(row)

    # Close cursor and connection
    cursor.close()
    conn.close()

except Exception as e:
    print("Error:", e)

(1, 'Alice', 'Johnson', 'alice.johnson@example.com', '123-456-7890', datetime.date(2022, 3, 15), 1, 1, Decimal('75000.00'))
(2, 'Bob', 'Smith', 'bob.smith@example.com', '234-567-8901', datetime.date(2021, 7, 10), 2, 2, Decimal('105000.00'))
(3, 'Charlie', 'Brown', 'charlie.brown@example.com', '345-678-9012', datetime.date(2023, 1, 20), 3, 3, Decimal('80000.00'))


In [1]:
import sqlite3
print(sqlite3.sqlite_version)


3.43.1
